This notebook divides Planet mosaic images into image chunks of 256 by 256 pixels as required by the crop boundary deep learning model. Chunks that are outside the AOIs or contain no cropped areas according to the DE Africa Cropland Extent Map (2019) are not saved.

## Load packages and modules

In [1]:
from glob import glob
import os
import subprocess
import rasterio
import datacube
import numpy as np
import geopandas as gpd
from shapely.geometry import box

/usr/local/lib/python3.10/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_12051/4292342901.py:7: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (callin

## Set parameters

In [2]:
# input shapefile of AOI used for downloading Planet images
# AOI_path='input_data/Rwanda_Boundary.shp'
AOI_path='input_data/Rwanda_test_ROI_finetune.shp'

# input folder for the mosaic images
input_folder='results'

# output folder to store image chunks
out_folder=input_folder+'/RGB_chunks'

# country/region name
country='Rwanda'

# year of data
str_year='2021'

# tiling parameters
tile_size=256 # chunk size

overlap=4 # number of overlapping pixels between chunks

In [3]:
if not os.path.isdir(out_folder):
    os.makedirs(out_folder)

## Clip into chunks and ignore those totally outside AOI or with no crops

In [4]:
images=glob(input_folder+'/'+country+'*_mosaic_clipped.tif')# search files
print('found {} mosaic images'.format(len(images)))

found 6 mosaic images


In [5]:
AOIs=gpd.read_file(AOI_path)
AOIs

,id,geometry
0,NaN,"POLYGON ((3365733.406 -181336.951, 3365733.406..."


- Extract chunk window (starting from left-top coner of mosaic image)
- Query DE Africa crop mask within window only when chunk window intersects AOI
- Export chunk as RGB bands raster only when crop exists within chunk

In [6]:
%%time
for image in images: # loop through all monthly mosaics
    print('clipping file',image,'into chunks...')
    
    # read in and get information of the mosaic
    ds = rasterio.open(image)
    crs=ds.crs.to_string()
    # x_min,y_min,x_max,y_max=ds.bounds
    width,height=ds.width,ds.height
    AOIs=AOIs.to_crs(crs)
    
    id_column=0 # record column id 
    for column_start in range(0, width-tile_size+overlap, tile_size-overlap):
        id_column+=1
        id_row=0 # record row id
        for row_start in range(0, height-tile_size+overlap, tile_size-overlap):
            id_row+=1
            
            # calculate clipping window
            row_end,column_end=row_start+tile_size,column_start+tile_size

            # load DE Africa crop mask 2019
            x_min,y_max=ds.xy(row_start,column_start)
            x_max,y_min=ds.xy(row_end,column_end)
            
            # get chunk bounding box
            chunk_geom=box(x_min,y_min,x_max,y_max)

            for index, row in AOIs.iterrows():# only do clipping when quad intersects with AOI
                AOI=AOIs.iloc[[index]]
                if chunk_geom.intersects(AOI.geometry.iloc[0]):
#                     print('intersecting')
                    dc = datacube.Datacube(app='cropland_extent')
                    query = {
                        'time': ('2019'),
                        'x': (x_min,x_max),
                        'y': (y_min,y_max),
                        'resolution':(-10, 10),
                        'crs':crs,
                    }
                    cm = dc.load(product='crop_mask',**query).squeeze()
                    np_crop_mask=cm['mask'].to_numpy()

                    # only do clipping and exporting if crop exists in the chunk
                    if np.nansum(np_crop_mask)>0:
                        # NOTE: increase zfill when needed
                        outname=os.path.basename(image)
                        outname=outname[:outname.index('_mosaic_clipped')]
                        outname=os.path.join(out_folder,'_'.join([outname,str(id_row).zfill(3),str(id_column).zfill(3)])+'.tif')
                        if not os.path.exists(outname):
                            # print('writing to file ',outname)
                            gdal_cmd=["gdal_translate", "-of", "GTiff", "-b","1","-b","2","-b","3",'-co','COMPRESS=DEFLATE',
                            "-srcwin",str(column_start),str(row_start),str(tile_size),str(tile_size),image,outname]
                            subprocess.call(gdal_cmd,stdout=subprocess.DEVNULL)
                    break
    ds=None

clipping file results/Rwanda_planet_medres_visual_2021_11_mosaic_clipped.tif into chunks...
clipping file results/Rwanda_planet_medres_visual_2021_04_mosaic_clipped.tif into chunks...
clipping file results/Rwanda_planet_medres_visual_2021_08_mosaic_clipped.tif into chunks...
clipping file results/Rwanda_planet_medres_visual_2021_12_mosaic_clipped.tif into chunks...
clipping file results/Rwanda_planet_medres_visual_2021_03_mosaic_clipped.tif into chunks...
clipping file results/Rwanda_planet_medres_visual_2021_10_mosaic_clipped.tif into chunks...
CPU times: user 14min 14s, sys: 4min 9s, total: 18min 24s
Wall time: 29min 10s


In [7]:
# # test if mosaic correctly removes unwanted chunks
# ! gdal_merge.py -o ../experiments/Mozambique_RGB/clipped/mosaic_test.tif -co COMPRESS=Deflate ../experiments/Mozambique_RGB/clipped/*.tif